<a href="https://colab.research.google.com/github/chandra-prakash-94/Langchain/blob/main/local_Chatting_with_Flan20B_UL2_using_LangChain_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install huggingface_hub langchain transformers

In [ ]:
!pip show langchain

In [3]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""

In [4]:
from langchain.llms import HuggingFaceHub

In [5]:
flan_ul2 = HuggingFaceHub(
    repo_id="google/flan-ul2",
    model_kwargs={"temperature":0.1,
                  "max_new_tokens":256})

In [6]:
flan_t5 = HuggingFaceHub(
    repo_id="google/flan-t5-xl",
    model_kwargs={"temperature":0 }
)

In [7]:
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

In [8]:
memory = ConversationBufferMemory()

In [9]:
conversation = ConversationChain(
    llm=flan_ul2,
    verbose=True,
    memory=memory
)

In [10]:
conversation.predict(input="Hi there! I am Sam")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there! I am Sam
AI:

> Finished chain.


'Hi, I am Sam. How can I help you?'

In [12]:
conversation.predict(input="How are you today?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there! I am Sam
AI: Hi, I am Sam. How can I help you?
Human: How are you today?
AI: I am good.
Human: How are you today?
AI:

> Finished chain.


'I am good.'

In [13]:
conversation.predict(input=" Can you help me with some customer support?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there! I am Sam
AI: Hi, I am Sam. How can I help you?
Human: How are you today?
AI: I am good.
Human: How are you today?
AI: I am good.
Human:  Can you help me with some customer support?
AI:

> Finished chain.


'Yes, I can help you with customer support.'

In [14]:
conversation.predict(input="My TV is broken. can you help fix it?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there! I am Sam
AI: Hi, I am Sam. How can I help you?
Human: How are you today?
AI: I am good.
Human: How are you today?
AI: I am good.
Human:  Can you help me with some customer support?
AI: Yes, I can help you with customer support.
Human: My TV is broken. can you help fix it?
AI:

> Finished chain.


'Yes, I can help you with that.'

## Counting the tokens

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("google/flan-ul2")

In [16]:
test_input = "Now Good Morning Ms Rogers"
# tokenizer([test_input])
tokenizer.tokenize(test_input)

['▁Now', '▁Good', '▁Morning', '▁M', 's', '▁Roger', 's']

In [17]:
conversation.memory.buffer

'Human: Hi there! I am Sam\nAI: Hi, I am Sam. How can I help you?\nHuman: How are you today?\nAI: I am good.\nHuman: How are you today?\nAI: I am good.\nHuman:  Can you help me with some customer support?\nAI: Yes, I can help you with customer support.\nHuman: My TV is broken. can you help fix it?\nAI: Yes, I can help you with that.'

In [18]:
formatted_prompt = conversation.prompt.format(input='the next input',history=memory.buffer)
formatted_prompt

'The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\nHuman: Hi there! I am Sam\nAI: Hi, I am Sam. How can I help you?\nHuman: How are you today?\nAI: I am good.\nHuman: How are you today?\nAI: I am good.\nHuman:  Can you help me with some customer support?\nAI: Yes, I can help you with customer support.\nHuman: My TV is broken. can you help fix it?\nAI: Yes, I can help you with that.\nHuman: the next input\nAI:'

In [19]:
def chat_to_llm(chat_llm ):
    conversation_total_tokens = 0
    new_conversation = ConversationChain(llm=chat_llm,
                                     verbose=False,
                                     memory=ConversationBufferMemory()
                                     )

    while True:
        message = input("Human: ")
        if message=='exit':
            print(f"{conversation_total_tokens} tokens used in total in this conversation")
            break
        if message:
            formatted_prompt = conversation.prompt.format(input=message,history=new_conversation.memory.buffer)
            num_tokens = len(tokenizer.tokenize(formatted_prompt))
            conversation_total_tokens += num_tokens
            print(f'tokens sent {num_tokens}')
            response = new_conversation.predict(input=message)
            response_num_tokens = len(tokenizer.tokenize(response))
            conversation_total_tokens += response_num_tokens
            print(f"LLM: {response}")




In [20]:
chat_to_llm(flan_ul2)

Human: Hi
tokens sent 60
LLM: Hi, how can I help you?
Human: what is the capital of India?
tokens sent 79
LLM: Delhi is the capital of India
Human: Thank You
tokens sent 91
LLM: Is there anything else I can help you with?
Human: No
tokens sent 107
LLM: Is there anything else I can help you with?
Human: No
tokens sent 123
LLM: Is there anything else I can help you with?


KeyboardInterrupt: Interrupted by user

In [ ]:
chat_to_llm(flan_t5)


Human: hi how are you?
tokens sent 64
LLM: I am good thanks.
Human: What is your name?
tokens sent 78
LLM: I am a robot.
Human: exit
153 tokens used in total in this conversation


## Using a Summary Memory

In [ ]:
from langchain.chains.conversation.memory import ConversationSummaryMemory

In [ ]:
summary_memory = ConversationSummaryMemory(llm=flan_ul2)

conversation = ConversationChain(
    llm=flan_ul2,
    verbose=True,
    memory=summary_memory
)

In [ ]:
conversation.predict(input="Hi there! I am Sam")

In [ ]:
conversation.predict(input="How are you today?")

In [ ]:
conversation.predict(input="I am good. What is your name?")

In [ ]:
conversation.predict(input="Thats cool can you help me with customer support?")

In [ ]:
conversation.predict(input="What time is your shop open today?")